In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import RandomForestRegressor

 
class MyStacking:
    def __init__(self, n_folds = 3, estimators = None, random_state = None, teacher_algorithm = RandomForestRegressor(n_jobs=-1, )):
        self.folds = n_folds
        self.estimators = estimators
        self.random_state = random_state
        self.alg_weight = None
        self.teacher_algorithm = teacher_algorithm
        self.fit_algorithm = []
    
    def fit(self, X, y):
        np.random.seed(self.random_state)

      # Создаем словарь, который будет хранить предсказанные данные
        meta_features = np.array([])

      # Обучаем наши алгоритмы для получения метапризнаков чтобы впоследствии обучить teacher_algorithm
        for name_alg, algorithm in self.estimators:
          # Создаю болванку для записи  предсказанных данных - pred_elem
          # group_elem - разбиение индексов на фолды - необходимо для кросс-предсказания
            pred_val = np.zeros((y.shape[0]))
            shuffle_ind = np.random.permutation(y.shape[0]) 
            group_elem = np.array_split(shuffle_ind, self.folds)

            for i, indicies in enumerate(group_elem):
                if i == 0 or i == (self.folds - 1):
                    train_indicies = np.hstack((group_elem[i+1:])) if i== 0 else np.hstack((group_elem[: i]))
                else:
                    train_indicies = np.hstack((np.hstack((group_elem[i+1:])), np.hstack((group_elem[:i]))))
                
                algorithm.fit(X[train_indicies, :], y[train_indicies])
                pred_val[indicies] = algorithm.predict(X[indicies, :])
            

            if meta_features.shape[0] == 0:
                meta_features = pred_val
            else:
                meta_features = np.vstack((meta_features, pred_val))

        meta_features = meta_features.T  
        self.teacher_algorithm.fit(meta_features, y)

        self.alg_weight =  self.teacher_algorithm.feature_importances_

        for name_alg, algorithm in self.estimators:
            self.fit_algorithm.append((name_alg, algorithm.fit(X, y)))
        

    def predict(self, X):
        meta_features = np.array([])
        for name_alg, algorithm in self.fit_algorithm:
            pred_val = algorithm.predict(X) 
            if meta_features.shape[0] == 0:
                meta_features = pred_val
            else:
                meta_features = np.vstack((meta_features, pred_val))
        
        print(meta_features)
        result = meta_features.T.dot(self.alg_weight)

        return result   


In [ ]:
hard_student = MyStacking(n_folds=3, estimators=[("sgd_reg", SGDRegressor()), ("svm_reg", SVR()), ("RF_reg", RandomForestRegressor())])
hard_student.fit(X_norm, y)

In [ ]:
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([
          ("std", StandardScaler()),
          ("PCA", PCA())
])

X_norm = pipe.fit_transform(X)

In [ ]:
X_norm.shape

(506, 13)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

hard_student = MyStacking(n_folds=3, estimators=[("sgd_reg", SGDRegressor()), ("svm_reg", SVR()), ("RF_reg", RandomForestRegressor())])
hard_student.fit(X_norm, y)

In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = hard_student.predict(X_norm)

mean_absolute_error(y_pred, y)

[[30.32910225 24.95418751 30.55574912 ... 27.68048068 26.17903084
  22.36525113]
 [29.32824737 23.06856722 31.22595594 ... 24.5041957  23.46038865
  20.14376826]
 [23.754      22.125      32.688      ... 24.021      22.598
  16.746     ]]


2.1353614947578534